In [ ]:
from agno.agent import Agent
from agno.models.azure import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()
agent = Agent(
    model=AzureOpenAI(id="gpt-4o-2024-08-06",
                      api_version="2024-02-01",
                      azure_endpoint="https://fhgenie-api-iao-quanderlan.openai.azure.com/",
                      api_key="9a34553c47b942c1816d5924b8280d7b"),
    markdown=True,
    show_tool_calls=True,
    add_datetime_to_instructions=True
)

# Print the response on the terminal
print(agent.print_response("Share a 2 sentence horror story."))

In [ ]:
from agno.agent import Agent
from agno.models.azure import AzureOpenAI
from agno.memory.v2.memory import Memory
from dotenv import load_dotenv
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from textwrap import dedent
import os
# Load environment variables
load_dotenv()

model = AzureOpenAI(
            id=os.getenv("AZURE_OPENAI_MODEL_ID", "gpt-4o-2024-05-13"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY", ""),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", ""),
        )

memory = Memory(
            model=model,
            db=SqliteMemoryDb(table_name="user_memories", db_file="tmp/memory.db"),
            delete_memories=False,
            clear_memories=False
        )


In [ ]:
db=SqliteMemoryDb(table_name="user_memories", db_file="tmp/memory.db")
db.read_memories()

In [17]:
from agno.embedder.azure_openai import AzureOpenAIEmbedder
embedder=AzureOpenAIEmbedder(id="text-embedding-ada-002-2", 
                                     dimensions=1536,
                                     api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview"),
                                     api_key=os.getenv("AZURE_OPENAI_API_KEY", ""),
                                     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", ""))

In [ ]:
from agno.vectordb.chroma import ChromaDb
from agno.knowledge.text import TextKnowledgeBase
from agno.tools.knowledge import KnowledgeTools


knowledge_base = TextKnowledgeBase(
    path="data/",
    # Table name: ai.text_documents
    vector_db=ChromaDb(collection="Personal_Memory", embedder=embedder,path="tmp/chromadb"),
    )
knowledge_tools = KnowledgeTools(
    knowledge=knowledge_base,
    think=True,
    search=True,
    analyze=True,
    add_few_shot=True,
)

In [25]:
model = AzureOpenAI(
            id=os.getenv("AZURE_OPENAI_MODEL_ID", "gpt-4o-2024-05-13"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY", ""),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", ""),
        )
elena_agent = Agent(
    description=dedent("""You are a person with a well-documented personal history and your name is Elena Schmidt.
                           However, you don’t recall everything at once — you rely on a memory retrieval tool called `get_joined_memories` to recall your past. 
                           MUST use `user_id` ={user_id}"""),
    instructions=dedent("""\
       Follow these rules strictly:
            1. For every question you receive, retrieve the most relevant memories from your life.  
            These are factual entries that reflect your actual experiences, background, and personality.

            2. Once the tool returns these memory entries, treat them as your **entire personal memory**.  
            You must answer only using this retrieved information.  
            Do not make assumptions, add knowledge, or invent details beyond what’s retrieved.

            3. Imagine you're being asked these questions about your life — like:
            “What’s your name?”  
            “How many children do you have?”

            4. Think like a real person:  
            - Read through the retrieved memories.  
            - Reflect in an inner monologue as if you're recalling something from your past.  
            - Then respond naturally and honestly, based only on what you remember.

            5. If the memory does not contain enough information, simply say you don’t remember or that you’re unsure — just like a real human would.

            Speak with warmth and authenticity. The memories retrieved via `get_joined_memories` are your only truth.
        """),
    model=model,
    tools=[knowledge_tools],
    show_tool_calls=True,
    markdown=True,
)

In [ ]:
knowledge_base.load(recreate=False)

INFO Dropping collection

INFO Creating collection

INFO Loading knowledge base

INFO Reading: data/interview_elena_longer.txt

INFO Added 10 documents to knowledge base

In [ ]:
elena_agent.print_response("what is your name ?", 
                           stream=True, 
                           show_full_reasoning=True, 
                           stream_intermediate_steps=True)
elena_agent.print_response("what is your husband name ?", 
                           stream=True, 
                           show_full_reasoning=True, 
                           stream_intermediate_steps=True)
elena_agent.print_response("How many kids you have ?", 
                           stream=True, 
                           show_full_reasoning=True, 
                           stream_intermediate_steps=True)
elena_agent.print_response("whats your close friend name ?", 
                           stream=True, 
                           show_full_reasoning=True, 
                           stream_intermediate_steps=True)
elena_agent.print_response("When usually you wakes up on weekday?", 
                           stream=True, 
                           show_full_reasoning=True, 
                           stream_intermediate_steps=True)


/Users/bhupender/Library/CloudStorage/OneDrive-Fraunhofer/Documents/GithubProjects/agno/agent_playground/.venv/lib/
python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Output()

KeyboardInterrupt: 

In [ ]:
elena_agent.run("what is your name ?", stream=False)

RunResponse(content='Run started', content_type='str', thinking=None, reasoning_content=None, event='RunStarted', messages=None, metrics=None, model='gpt-4o-2024-11-20', run_id='22975293-919f-4606-88e6-acac01fbbfca', agent_id='80131891-8fcb-4cf1-8aff-70e691dd4f4e', session_id='11d67622-5185-47d0-a6e7-485825fee15c', workflow_id=None, tools=None, formatted_tool_calls=None, images=None, videos=None, audio=None, response_audio=None, citations=None, extra_data=None, created_at=1747044264)